<a href="https://colab.research.google.com/github/morrijm4/Calculator/blob/master/Assignment_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment we are going to work with data that was downloaded using using the [yfinance](https://pypi.org/project/yfinance/) library. It containst daily closing prices of all symbols.

In [1]:
from multiprocessing   import Pool
from tqdm import *
import pandas as pd

In [2]:
#Question 1: Explain what the code below is trying to do. Describe each step.
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())


In [3]:
#Describe your answer below.
#Answer 1: Reads data from csv's and puts the data without test issues into sets.

In [4]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

--2022-01-26 18:25:00--  https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/gzpi950gfdz9dsk/stocks.parquet [following]
--2022-01-26 18:25:00--  https://www.dropbox.com/s/dl/gzpi950gfdz9dsk/stocks.parquet
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4cc5c3e86022e69c862d114d19.dl.dropboxusercontent.com/cd/0/get/BejHj50GxjiRqehpBbi0zxfDNWQNSAb4sgrFFE0_ncxrfR4pU3w6VRMcbp1QjVkcJSAbINx3W-XzI1Mh834spDOudtjOKj_1zQCMkYWJNJxP_QmgRaJyK9uVfCGQBnfLOwUrcdkyE4ZSY8dbbjEaRXaB/file?dl=1# [following]
--2022-01-26 18:25:00--  https://uc4cc5c3e86022e69c862d114d19.dl.dropboxusercontent.com/cd/0/get/BejHj50GxjiRqehpBbi0zxfDNWQNSAb4sgrFFE0_ncxrfR4pU3w6VRMcbp1QjVkcJSAbINx3W-XzI1Mh83

In [5]:
!ls stocks.parquet -l #downloaded file

-rw-rw-r-- 1 ubuntu ubuntu 678640589 Jan 26 18:25 stocks.parquet


In [6]:
#Bonus question 1 - find on internet and write the command that will download the all data from the symbols list. 

In [7]:
data=pd.read_parquet('stocks.parquet')

In [8]:
data.head(1) # Below shows a sample of the data.
#date - describes the day of the price
#open - the price at which stock opened
#high - what was the intraday high
#low - what is the intraday low
#close - what was the closing price
#adj_close - the price calculated after adjusting for all future stock splits
#dividends - what are the dividends that were paid on the day.
#stock_splits - if the stock was split
#symbol the symbol of the stock


,date,open,high,low,close,adj_close,volume,dividends,stock_splits,symbol,day,year,month,dayofweek
0,1962-01-02,0.131273,0.131273,0.124177,0.124177,0.045278,2480333.0,0.0,0.0,HPQ,2,1962,1,Tuesday


In [ ]:
# Question 2 find top 20 best performer stocks - i.e. the stocks that provided the best return.
# if the stocks adj_close on the earliest date was X and the adj_close on the 2021-01-01 is y and the number of days elapsed between these dates is Z.
# Then return = Y/X/Z
# Sort the stocks by this return and find the top 20

data_symbols = data['symbol'].unique()
data['date'] = pd.to_datetime(data['date'], format="%Y-%m-%d")
returns = []

for symbol in tqdm(data_symbols, desc="Loading..."):
  symbol_df = data[data['symbol'] == symbol]

  early = symbol_df.iloc[0]
  late = symbol_df.iloc[-1]

  returns.append((symbol, (late.get('adj_close') / early.get('adj_close')) / (late.get('date') - early.get('date')).days))
  
rankings = sorted(returns, key = lambda x: x[1], reverse=True)

for i in range(20):
  print(rankings[i])

In [ ]:
#Question 3: Create a copy of this notebook called assignment-aws and follow the instructions in readme to run the notebook on AWS compute instance.
# Compare performance of running natively on colab and colab+AWS